In [1]:
!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached albumentations-1.3.0-py3-none-any.whl (123 kB)
  Using cached qudida-0.0.4-py3-none-any.whl (3.5 kB)
  Using cached opencv_python_headless-4.7.0.72-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (49.2 MB)


# Data Processsing

In [18]:
train_dir = '/home/ec2-user/SageMaker/train/'
test_dir = '/home/ec2-user/SageMaker/test/'
files_dir = '/home/ec2-user/SageMaker/'

## Labels

In [19]:
import json

with open(files_dir + '/label_num_to_disease_map.json', 'r') as f:
    labels_info = json.load(f)

NUM_LABELS = len(labels_info)
print(labels_info)

{'0': 'Cassava Bacterial Blight (CBB)', '1': 'Cassava Brown Streak Disease (CBSD)', '2': 'Cassava Green Mottle (CGM)', '3': 'Cassava Mosaic Disease (CMD)', '4': 'Healthy'}


## Load Data with Pytorch

In [20]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv(files_dir + '/train.csv')

# Split the data into train, validation, and test sets with a ratio of 80:10:10
train_df, val_test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

## regular images (non-transformed)

In [22]:
class CassavaDataset(Dataset):
    def __init__(self, data, root_dir, transform=None):
        self.image_labels = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.image_labels.iloc[index, 0])
        image = Image.open(img_path).convert('RGB')
        label = self.image_labels.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

## augmented images

In [23]:
# import albumentations as A
# from albumentations.pytorch import ToTensorV2
# import cv2

# class CassavaDataset(Dataset):
#     def __init__(self, data, root_dir, transform=None):
#         self.image_labels = data
#         self.root_dir = root_dir
#         self.transform = transform

#         # add CLAHE augmentation
#         self.clahe_transform = A.Compose([
#             A.CLAHE(always_apply = True),
#             ToTensorV2()
#         ])

#     def __len__(self):
#         return len(self.image_labels)

#     def __getitem__(self, index):
#         img_path = os.path.join(self.root_dir, self.image_labels.iloc[index, 0])
#         image = Image.open(img_path).convert('RGB')
#         label = self.image_labels.iloc[index, 1]
        
#         # apply CLAHE augmentation
#         image = self.clahe_transform(image=np.array(image))['image']
        
#         if self.transform:
#             image = self.transform(image)
#         return image, label

# Useful Functions

In [24]:
# Save the model so that we don't need to train the same model multiple times
def save_model(model, model_name):
  torch.save(model.state_dict(),  files_dir + 'saved_models/' + model_name + '.pth')

# Training with Resnet

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


In [26]:
from torchvision.models import resnet50, ResNet50_Weights

In [27]:
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()

In [28]:
transform = transforms.Compose([
    preprocess
])

In [29]:
train_dataset = CassavaDataset(train_df, train_dir, transform=transform)
validation_dataset = CassavaDataset(val_df, train_dir, transform=transform)
test_dataset = CassavaDataset(test_df, train_dir, transform=transform)

In [30]:
# Define the batch size for the data loader
batch_size = 64
# Create the data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [31]:
# Pre-trained weights
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# Linear Layers
model.fc = torch.nn.Sequential(
    torch.nn.Linear(model.fc.in_features, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, NUM_LABELS)
)

# model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [45]:
# model_name = 'resnet50_5_epochs_512_batch'
model_name = 'resnet50_5_epochs_64_batch_augment'
model.load_state_dict(torch.load(files_dir + 'saved_models/' + model_name + '.pth'))
model.to(device)
# set the model to evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [46]:
# # Train
# import numpy as np
# EPOCHS = 5 # Each epoch takes around 16 minutes

# train_loss, validation_loss = [], []
# train_acc, validation_acc = [], []

# for epoch in range(EPOCHS):
#     print("Training Epoch: ", epoch)
#     batch_num = 0
    
#     model.train()
#     running_loss = 0.
#     correct, total = 0, 0 
    
#     for inputs, labels in train_loader:
#         batch_num += 1
#         print("Current Batch: ", batch_num)
#         # Forward pass
#         inputs = inputs.to(device)
#         labels = labels.to(device)
         
#         predictions = model(inputs)
#         loss = criterion(predictions, labels)

#         # Backward and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
            
#         _, predicted = torch.max(predictions, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
    
#     train_loss.append(running_loss / len(train_loader))
#     train_acc.append(correct/total)

#     model.eval()
#     running_loss = 0.
#     correct, total = 0, 0 
    
#     for inputs, labels in val_loader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         predictions = model(inputs)

#         loss = criterion(predictions, labels)

#         running_loss += loss.item()

#         _, predicted = torch.max(predictions, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     validation_loss.append(running_loss / len(val_loader))
#     validation_acc.append(correct/total)
    
#     print(f"Epoch {epoch+1}:")

#     print(f"Training Loss:", round(train_loss[epoch], 3))
#     print(f"Validation Loss:", round(validation_loss[epoch], 3))

#     print(f"Training Accuracy:", round(train_acc[epoch], 3))
#     print(f"Validation Accuracy:", round(validation_acc[epoch], 3))

#     print("------------------------------")

In [47]:
# save_model(model, "resnet50_5_epochs_64_batch_augment")

In [48]:
# Calculate the test accuracy 
import numpy as np
model.eval()

test_predictions = np.array([])
true_labels = np.array([])

total = 0
correct = 0

for inputs, labels in test_loader:
    
    inputs = inputs.to(device)
    labels = labels.to(device)

    predictions = model(inputs)
    
    _, predicted = torch.max(predictions, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
    true_labels = np.append(true_labels, labels.cpu().numpy())
    test_predictions = np.concatenate((test_predictions, predicted.detach().cpu().numpy()))
    
print("Test Accuracy: ", correct/total)

Test Accuracy:  0.7836448598130841


In [49]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

f1 = f1_score(true_labels, test_predictions, average='weighted')
recall = recall_score(true_labels, test_predictions, average='weighted')
precision = precision_score(true_labels, test_predictions, average='weighted')
accuracy = accuracy_score(true_labels, test_predictions)

# Print the accuracy, F1 score, recall, and precision
print("Test Accuracy: ", accuracy)
print("F1 Score: ", f1)
print("Recall: ", recall)
print("Precision: ", precision)

Test Accuracy:  0.7836448598130841
F1 Score:  0.7706074832573018
Recall:  0.7836448598130841
Precision:  0.8043222928281726


In [50]:
test_predictions

array([3., 3., 1., ..., 3., 3., 3.])

In [51]:
true_labels

array([3., 3., 1., ..., 3., 0., 3.])